In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6,
    WL_total=2.4,
    WL_residual=2.2
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7,
    WL_total=-1.7,
    WL_residual=-1.4
)


## Parameters

In [4]:
# General constants
g = 9.81 # m/s^2
rho_w = 1025 # kg/m^3
rho_s = 2650 # kg/m^3
nu = 1.33e-6 # m^2/s
bodemdiepte = 40 # mMSL
Kv = 1.4 # Velocity coefficient
Delta_v = (rho_s - rho_w) / rho_w # Dimensionless

## Armour Layer
### Boek methode

Diameter van de armour layer wordt hieronder bepaald via de methode boek: de adpated shield approach. Alleen snelheid (u_c) en waterdiepte (h) zullen varieren tussen weeromstandigheden.
- De waarde van Kv moet nog geverifieerd worden. Of bij docenten of uit literatuur 

In [5]:
#solve the dispersion relation using Newton-Raphson
def waveNumber_dispersionRelation_NewtonRaphson(T,d,tolerance=1e-6):
    grav=9.81
    dif=999999
    w=2*np.pi/T
    w2 = w*w
    # use deep-water wavenumber as starting value
    ko=w2/grav
    # j is iteration number
    j = 1
    while dif > tolerance:
        fk = w2 - grav*ko*np.tanh(ko*d)
        fak = -grav*(np.tanh(ko*d)+ko*d*(1/np.cosh(ko*d))**2)
        kn = ko - fk / fak
        dif=abs(kn-ko)
        ko = kn
        j = j + 1
    k=kn
    return k  # Return the final calculated wave number

In [6]:
def calculate_d_n50(u_c, H, T, h, psi_c, K_s=1.0, K_v=Kv, delta=Delta_v):
    """
    Calculate the characteristic stone diameter d_n50 for an armour layer (with adapted Shields formula).

    Parameters 
    ----------
    u_c : float
        Current velocity depth-averaged [m/s]
    H : float
        Wave height [m]
    T : float
        Wave period [s]
    h : float
        Water depth [m]
    psi_c : float, optional
        Critical motion parameter [-]
    K_s : float, optional
        Strength reduction factor for stones on a slope [-]
    K_v : float, optional
        Velocity/turbulence factor [-]
    delta : float, optional
        Relative density (Δ = (rho_s - rho_w) / rho_w) [-]

    Returns
    -------
    d_n50 : float
        Required median stone diameter [m]
    """
    # Wave characteristics
    omega = 2.0 * np.pi / T
    k = waveNumber_dispersionRelation_NewtonRaphson(T, h)  # Update k using dispersion relation

    # Bottom orbital velocity amplitude
    a_b = (H / 2.0) / np.sinh(k * h)
    u_b_hat = omega * a_b
    print(f'Wave parameters: T={T:.1f} s, L={2*np.pi/k:.0f} m, kd={k*h:.2f} a_b={a_b:.4f} m, u_b_hat={u_b_hat:.4f} m/s')
    # Shear velocity calculation
    C = 50.0  # initial Chezy coefficient guess
    k_r = 0.1  # initial roughness guess
    c_f = 0.237 * (a_b / k_r)**(-0.52) # initial friction factor guess
    print(f'Initial: C={C:.0f}, c_f={c_f:.4f}, k_r={k_r:.1f}')
    
    for i in range(1, 8):
        # Time-averaged shear velocity (equation 7.6, time-averaged)
        # u_star_r = np.sqrt((g / C**2) * u_c**2 + (c_f / 4) * u_b_hat**2) # time-averaged shear velocity
        u_star_r = np.sqrt((g / C**2) * u_c**2 + \
                                (c_f / 2) * u_b_hat**2 + \
                                2 * np.sqrt(g) / C * u_c * np.sqrt(c_f / 2) * u_b_hat * np.sin(np.pi / 2))  # Max instantanous shear velocity
        tau_r = rho_w * u_star_r**2
        d_n50 = (u_star_r**2) / (psi_c * delta * g)
        
        # Update roughness
        k_r = 2 * d_n50
        C = 18 * np.log10(12 * h / k_r)
        c_f = 0.237 * (a_b / k_r)**(-0.52)
        
        print(f'{i}: tau_r={tau_r:.1f} Pa, d_n50={d_n50 * 100:.3f} cm, u_star_r={u_star_r:.3f} m/s, c_f={c_f:.5f}, k_r={k_r:.5f}')
    
    d_n50 = (K_v**2 * u_star_r**2) / (K_s * psi_c * delta * g)
    return u_star_r, d_n50

res_HWL = calculate_d_n50(u_c=HWL.CS_total, H=HWL.Hm0, T=HWL.Tp, h=HWL.WL_total + bodemdiepte, psi_c=0.03)
print(round(res_HWL[1] * 100, 2), 'cm')

Wave parameters: T=15.3 s, L=274 m, kd=0.97 a_b=4.3695 m, u_b_hat=1.7944 m/s
Initial: C=50, c_f=0.0332, k_r=0.1
1: tau_r=70.7 Pa, d_n50=14.788 cm, u_star_r=0.263 m/s, c_f=0.05843, k_r=0.29575
2: tau_r=114.1 Pa, d_n50=23.850 cm, u_star_r=0.334 m/s, c_f=0.07491, k_r=0.47700
3: tau_r=144.9 Pa, d_n50=30.303 cm, u_star_r=0.376 m/s, c_f=0.08485, k_r=0.60606
4: tau_r=163.5 Pa, d_n50=34.180 cm, u_star_r=0.399 m/s, c_f=0.09033, k_r=0.68361
5: tau_r=173.7 Pa, d_n50=36.317 cm, u_star_r=0.412 m/s, c_f=0.09322, k_r=0.72635
6: tau_r=179.1 Pa, d_n50=37.445 cm, u_star_r=0.418 m/s, c_f=0.09472, k_r=0.74890
7: tau_r=181.9 Pa, d_n50=38.027 cm, u_star_r=0.421 m/s, c_f=0.09548, k_r=0.76053
74.53 cm


In [7]:
res_HWL = calculate_d_n50(u_c=HWL.CS_total, H=HWL.Hm0, T=HWL.Tp, h=HWL.WL_total + bodemdiepte, psi_c=0.03)
res_LWL = calculate_d_n50(u_c=LWL.CS_total, H=LWL.Hm0, T=LWL.Tp, h=LWL.WL_total + bodemdiepte, psi_c=0.03)
print(f'For HWL conditions:  \n\t- u_c = {HWL.CS_total} m/s,\n\t- H = {HWL.Hm0} m,\n\t- T = {HWL.Tp} s,\n\t- h = {HWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_HWL[0]:.4f} m/s\n and the required median stone diameter d_n50is:\t d_n50 = {res_HWL[1]*100:.3f} cm.\n')

print(f'For LWL conditions:  \n\t- u_c = {LWL.CS_total} m/s,\n\t- H = {LWL.Hm0} m,\n\t- T = {LWL.Tp} s,\n\t- h = {LWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_LWL[0]:.4f} m/s\n and the required median stone diameter d_n50is:\t d_n50 = {res_LWL[1]*100:.3f} cm.\n')

Wave parameters: T=15.3 s, L=274 m, kd=0.97 a_b=4.3695 m, u_b_hat=1.7944 m/s
Initial: C=50, c_f=0.0332, k_r=0.1
1: tau_r=70.7 Pa, d_n50=14.788 cm, u_star_r=0.263 m/s, c_f=0.05843, k_r=0.29575
2: tau_r=114.1 Pa, d_n50=23.850 cm, u_star_r=0.334 m/s, c_f=0.07491, k_r=0.47700
3: tau_r=144.9 Pa, d_n50=30.303 cm, u_star_r=0.376 m/s, c_f=0.08485, k_r=0.60606
4: tau_r=163.5 Pa, d_n50=34.180 cm, u_star_r=0.399 m/s, c_f=0.09033, k_r=0.68361
5: tau_r=173.7 Pa, d_n50=36.317 cm, u_star_r=0.412 m/s, c_f=0.09322, k_r=0.72635
6: tau_r=179.1 Pa, d_n50=37.445 cm, u_star_r=0.418 m/s, c_f=0.09472, k_r=0.74890
7: tau_r=181.9 Pa, d_n50=38.027 cm, u_star_r=0.421 m/s, c_f=0.09548, k_r=0.76053
Wave parameters: T=13.4 s, L=223 m, kd=1.08 a_b=3.3335 m, u_b_hat=1.5631 m/s
Initial: C=50, c_f=0.0383, k_r=0.1
1: tau_r=62.8 Pa, d_n50=13.133 cm, u_star_r=0.248 m/s, c_f=0.06323, k_r=0.26265
2: tau_r=95.2 Pa, d_n50=19.905 cm, u_star_r=0.305 m/s, c_f=0.07849, k_r=0.39811
3: tau_r=117.1 Pa, d_n50=24.496 cm, u_star_r=0.338

### ARMOUR LAYER  HASPRO method



In [8]:
d_50 = 0.01 # m 
D_p = 9 # m
g = 9.81 # m/s^2
rho_w = 1025 # kg/m^3
rho_stone = 2650 # kg/m^3 [from stone grading table]
nu = 1.33e-6 # m^2/s
H_s = 9.9 # m
T_p = 10.5 # s
u_c = 0.5 # m/s
h_w = 40 # m [water depth at site]
D_n50 = d_50 * 0.84 # m 
n = 2.5 # given constant


def calc_hydrodynamic_params(T_p, H_s, u_c, h_w, D_n50, D_p, g=9.81):
    T_z = T_p / 1.3 

    u_m_bed = np.sqrt(2) * (H_s / 4) * np.sqrt(g / h_w) * np.exp(-((3.65 / T_z) * np.sqrt(h_w / g))**2.1)

    L_w_peak_period = 150
    for repetition in range(10):
        L_w_peak_period = g * T_p**2 / (2 * np.pi) * np.tanh((2 * np.pi * h_w) / L_w_peak_period)
    
    K_top =  np.sinh(2 * np.pi * h_w / L_w_peak_period) / np.sinh(2 * np.pi * (h_w - 4 * D_n50) / L_w_peak_period)

    u_m_top = u_m_bed * K_top

    A_w_a = (u_m_top * T_p) / (2 * np.pi)

    KC_c = np.abs(u_c) * T_p / D_p 
    KC_w = u_m_bed * T_p / D_p 
    KC_total = KC_c + KC_w

    return u_m_bed, u_m_top, A_w_a, KC_c, KC_w, KC_total , T_z, L_w_peak_period



In [9]:
def bed_shear_stresses(d_50, n, A_w_a, u_m_top, rho_w, u_c, h_w):
    k_s = n * d_50

    ampl_ratio = A_w_a / k_s

    wave_friction_coefficient = 0.237 * (ampl_ratio)**(-0.52)

    u_ww = np.sqrt(wave_friction_coefficient / 2) * u_m_top

    tau_bed_wave = rho_w * u_ww**2

    bed_roughness_length = k_s / 30

    drag_coefficient = (0.4 / (np.log(h_w / bed_roughness_length) - 1))**2

    shear_velocity_current = np.sqrt(drag_coefficient) * u_c

    tau_bed_current = rho_w * shear_velocity_current**2

    tau_mean = tau_bed_current + 1.2 * tau_bed_current * (tau_bed_wave / (tau_bed_current + tau_bed_wave))**3.2

    tau_max = tau_mean + tau_bed_wave 

    return wave_friction_coefficient, tau_bed_wave, tau_bed_current, tau_mean, tau_max 




In [10]:
def mobility_number(rho_stone, rho_w, d_50, g, nu, tau_max):
    delta_s = (rho_stone - rho_w) / rho_w
    d_star = d_50 * (delta_s * g / nu**2)**(1/3)
    shields_crit = 0.30 / (1 + 1.2 * d_star) + 0.055 * (1 - np.exp(-0.02 * d_star))
    shields_combined = tau_max / (delta_s * g * d_50 * rho_w)
    MOB_top = shields_combined / shields_crit

    return d_star, shields_crit, shields_combined, MOB_top


In [11]:
def depth_deformation(KC_total, D_p, MOB_top):
    
    f_ = 1 + (3.9274 / (1 + np.exp(-0.7401 * KC_total + 4.7518)))
    s_50 = D_p * f_ * 0.0707 * MOB_top**1.6492
    s_90 = D_p * f_ * 0.1134 * MOB_top**1.6492

    return f_, s_50, s_90




In [12]:
def acceptable_s(KC_total, acceptable_movement):
    if KC_total <= 4.2:
        if acceptable_movement == 'Very limited':
            s_acc_50 = 0.48
            s_acc_90 = 0.34
        elif acceptable_movement == 'Limited':
            s_acc_50 = 0.715
            s_acc_90 = 0.5
        elif acceptable_movement == 'Significant':
            s_acc_50 = 0.895
            s_acc_90 = 0.63
        elif acceptable_movement == 'Extreme':
            s_acc_50 = 1.055
            s_acc_90 = 0.735
    else:
        if acceptable_movement == 'Very limited':
            s_acc_50 = 0.36
            s_acc_90 = 0.235
        elif acceptable_movement == 'Limited':
            s_acc_50 = 0.535
            s_acc_90 = 0.355
        elif acceptable_movement == 'Significant':
            s_acc_50 = 0.675
            s_acc_90 = 0.45
        elif acceptable_movement == 'Extreme':
            s_acc_50 = 0.8
            s_acc_90 = 0.53
    return s_acc_50, s_acc_90


    

In [13]:
acceptable_movement = 'Very limited' # options: 'Very limited', 'Limited', 'Significant', 'Extreme'

def amour_layer_computation(acceptable_movement=acceptable_movement):

    'Computation makes use of parameters chosen by user'

    d_50 = 0.1 # m 
    D_p = 9 # m
    g = 9.81 # m/s^2
    rho_w = 1025 # kg/m^3
    rho_stone = 2650 # kg/m^3 [from stone grading table]
    nu = 1.33e-6 # m^2/s
    H_s = 9.9 # m
    T_p = 15.4 # s
    u_c = 0.5 # m/s
    h_w = 40 # m [water depth at site]
    D_n50 = d_50 * 0.84 # m 
    n = 2.5 # given constant

    

    for repetition in range(1000):

        u_m_bed, u_m_top, A_w_a, KC_c, KC_w, KC_total, T_z, L_w_peak_period = calc_hydrodynamic_params(HWL.Tp, HWL.Hm0, HWL.CS_total, HWL.WL_total + bodemdiepte, D_n50, D_p)

        wave_friction_coefficient, tau_bed_wave, tau_bed_current, tau_mean, tau_max = bed_shear_stresses(d_50, n, A_w_a, u_m_top, rho_w, HWL.CS_total, HWL.WL_total + bodemdiepte)

        d_star, shields_crit, shields_combined, MOB_top = mobility_number(rho_stone=rho_stone, rho_w=rho_w, d_50=d_50, g=g, nu=nu, tau_max=tau_max)

        f_, s_50, s_90 = depth_deformation(KC_total=KC_total, D_p=D_p, MOB_top=MOB_top)

        s_acc_50, s_acc_90 = acceptable_s(KC_total = KC_total, acceptable_movement=acceptable_movement)

        print(f's_50 = {s_50:.3f} m, s_acc_50 = {s_acc_50} m, | s_90 = {s_90:.3f} m, s_acc_90 = {s_acc_90} m')
    
        if s_50 <= s_acc_50 and s_90 <= s_acc_90:
            print(f'\nDesign is acceptabed with d_50 = {d_50:.3f} m after {repetition} iterations.')
            break
        else:
            d_50 += 0.01
            D_n50 = d_50 * 0.84 # update D_n50 based on new d_50
            print(f'Increasing d_50 to {d_50:.3f} m and dn50 to {D_n50:.3f} m for next iteration.')
    
    return d_50, s_50, s_90, s_acc_50, s_acc_90, MOB_top , shields_crit

d_50_final, s_50_final, s_90_final, s_acc_50_final, s_acc_90_final, MOB_top, shields_crit = amour_layer_computation()

print(f'\nThe maximum deformation allowed for the chosen acceptable movement "{acceptable_movement}" are:\ns_acc_50 = {s_acc_50_final} m\ns_acc_90 = {s_acc_90_final} m')
print(f'\nFinal design:\nd_50 = {d_50_final:.3f} m\ns_50 = {s_50_final:.3f} m\ns_90 = {s_90_final:.3f} m\ns_acc_50 = {s_acc_50_final} m\ns_acc_90 = {s_acc_90_final} m')
print(f'\nMobility number MOB_top = {MOB_top:.3f}')
print('\n--- END OF CALCULATION ---')



s_50 = 0.281 m, s_acc_50 = 0.48 m, | s_90 = 0.451 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.110 m and dn50 to 0.092 m for next iteration.
s_50 = 0.261 m, s_acc_50 = 0.48 m, | s_90 = 0.419 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.120 m and dn50 to 0.101 m for next iteration.
s_50 = 0.244 m, s_acc_50 = 0.48 m, | s_90 = 0.392 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.130 m and dn50 to 0.109 m for next iteration.
s_50 = 0.230 m, s_acc_50 = 0.48 m, | s_90 = 0.368 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.140 m and dn50 to 0.118 m for next iteration.
s_50 = 0.217 m, s_acc_50 = 0.48 m, | s_90 = 0.348 m, s_acc_90 = 0.34 m
Increasing d_50 to 0.150 m and dn50 to 0.126 m for next iteration.
s_50 = 0.206 m, s_acc_50 = 0.48 m, | s_90 = 0.330 m, s_acc_90 = 0.34 m

Design is acceptabed with d_50 = 0.150 m after 5 iterations.

The maximum deformation allowed for the chosen acceptable movement "Very limited" are:
s_acc_50 = 0.48 m
s_acc_90 = 0.34 m

Final design:
d_50 = 0.150 m
s_50 = 0.206 m
s_90 = 0